In [2]:
import pandas as pd
import json
#from unidecode import unidecode

In [9]:
# Specify the Excel file path
excel_file_path = '../data/raw/disreg_enceursalud20_a.xlsx'

# List of sheet names
sheet_names = ['Tablas1', 'Tablas2', 'Tablas3', 'Tablas4']

In [10]:
# Initialize variables
tables = {}

# Iterate through sheets
for sheet_name in sheet_names:
    # Read the Excel sheet and drop the specified column
    df = pd.read_excel(excel_file_path, sheet_name).drop('Unnamed: 2', axis=1, errors='ignore')

    # Initialize variables
    current_table = None
    table_data = {}

    # Iterate through rows
    for index, row in df.iterrows():
        # Check if the current row is the start of a new table
        if isinstance(row['Unnamed: 0'], str) and not row['Unnamed: 0'].isdigit() and row['Unnamed: 0'] != 'Código ':
            # Save the previous table data
            if current_table is not None:
                tables[current_table] = table_data

            # Set the current table name and initialize an empty dictionary for table data
            current_table, table_data = row['Unnamed: 0'], {}

        # Check if the row contains data (skip rows with NaN values)
        if pd.notna(row['Unnamed: 0']) and (isinstance(row['Unnamed: 0'], int) or (isinstance(row['Unnamed: 0'], str) and row['Unnamed: 0'].isdigit())):
            # Append the row data to the table_data dictionary
            table_data[row['Unnamed: 0']] = row['Unnamed: 1']

    # Save the last table data for the current sheet
    if current_table is not None:
        tables[current_table] = table_data

# Save the combined data to a JSON file
json_file_path = '../data/json_files/var_keys_dict.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(tables, json_file, indent=2, ensure_ascii=False)

print(f'Successfully converted {excel_file_path} to {json_file_path}')

Successfully converted ../data/raw/disreg_enceursalud20_a.xlsx to ../data/json_files/var_keys_dict.json


In [12]:
df = pd.read_excel(excel_file_path, 'Diseño', usecols=['Variable','Descripción','Diccionario de la variable']).set_index('Variable')

#renames the column with a shorter name
df.dropna(subset='Diccionario de la variable', inplace=True)
df.rename(columns={'Diccionario de la variable':'diccionario'}, inplace=True)

#converts the variable dictionary into json format
json_file_path = '../data/json_files/var_dict.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    df.to_json(json_file, orient='index', lines=False, indent=2,force_ascii=False, default_handler=str)

In [14]:
# Load JSON strings into dictionaries
path1 = '../data/json_files/var_keys_dict.json'
path2 = '../data/json_files/var_dict.json'

with open(path1, 'r') as json_file: #loads dictionary variables
    var_keys_dict = json.load(json_file)

with open(path2, 'r') as json_file: #loads json value keys of variables
    var_dict = json.load(json_file)

# Iterate through the first JSON (var_dict) and replace values
for key, value in var_dict.items():
    if "diccionario" in value and value["diccionario"] in var_keys_dict:
        var_dict[key]["diccionario"] = var_keys_dict[value["diccionario"]]

# Storing the resulting JSON (as full_var_dict)
json_file_path = '../data/json_files/full_var_dict.json'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(var_dict, json_file, indent=2, ensure_ascii=False)